## Imports

In [1]:
import geemap
import ee

# Initialize the Earth Engine module.
ee.Initialize()

## Feature & Image Collections

In [11]:
# Image Collections
nicfiColl = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia")
s2Coll = ee.ImageCollection("COPERNICUS/S2_SR")
s1Coll = ee.ImageCollection("COPERNICUS/S1_GRD")
dwColl = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# Feature Collections & Geometries
nrbc = ee.FeatureCollection("projects/ee-craigdsouza/assets/canals/nrbc_d9_d18")
nrbcBBox = nrbc.geometry().bounds().buffer(2000)
nrbcBuffer = nrbc.geometry().buffer(20)
geometry = ee.Geometry.Polygon(
        [[[76.35728463743025, 16.21064824268761],
          [76.35728463743025, 16.198944508521084],
          [76.38449296567732, 16.198944508521084],
          [76.38449296567732, 16.21064824268761]]], None, False);


## Visualization Params

In [ ]:
visParamsNICFI = {'bands':["R","G","B"],'min':400,'max':2000}
visParamsS2 = {'bands':["R","G","B"],'min':500,'max':3500}

ndwiPalette = ['fff','90bdf3','0600e8']
visParamsW = {'bands':['NDWI'],'palette':ndwiPalette,'min':-0.5,'max':0}

ndviPalette = ['f00','ff0','0f0']    #['EB2121','FBFF59','3CAE12']
visParamsV = {'bands':['NDVI'],'palette':ndviPalette,'min':0,'max':0.6}

dwPalette = ['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b','a59b8f', 'b39fe1']
dwVis = {'bands':['label'],'palette':dwPalette,'min':0,'max':8}

## Functions

In [3]:
def prepND(image):
  ndwi = image.normalizedDifference(["G","N"])
  ndvi = image.normalizedDifference(["N","R"])
  mask = dw.eq(4) #.or(dw.eq(5))
  aoi = nrbcBuffer
  ndwi = ndwi.clip(aoi).rename('NDWI') #.updateMask(mask)
  ndvi = ndvi.clip(aoi).rename('NDVI') #.updateMask(mask)
  return image.addBands([ndwi,ndvi]) #.updateMask(mask)

In [6]:
# Make NICFI Subset for one hydrological year
def makeNICFISubset(year):
  ndColl = []
  months = [6,7,8,9,10,11,12,1,2,3,4,5]
  for month in months:
    start = f"{year}" + "-" + f"{month:02d}" + "-" + "01"
    end = f"{year+1}" + "-" + f"{month:02d}" + "-" + "01"
    monthlyMosaic = nicfiColl.filterDate(start,end).first()
    ndMosaic = prepND(monthlyMosaic)
    ndColl.append(ndMosaic)
    print(f"{year}-{month:02d} prepared")
  return ee.ImageCollection(ndColl)

In [52]:
# Blend NDWI canal images with basemap  
def blendImages(image,basemap):
  # Add visualization parameters to your image
  ndwiPalette = ['fff','90bdf3','0600e8']
  visParamsW = {'bands':['NDWI'],'palette':ndwiPalette,'forceRgbOutput':True,'min':-0.5,'max':0}
  visualizedImage = image.visualize(**visParamsW).set('system:time_start',image.get('system:time_start'))
  # Blend your image with the basemap
  return basemap.blend(visualizedImage).set('system:time_start',image.get('system:time_start'))

## Prepare Images

In [5]:
# Dynamic World
dw = dwColl.filterDate('2022-06-01','2023-06-01').filterBounds(nrbcBBox).mode().select('label').clip(nrbcBBox)

In [ ]:
# NICFI Subset
nd_mosaic_2022 = makeNICFISubset(2022)
print(nd_mosaic_2022.size().getInfo()) # 12
print(nd_mosaic_2022.first().bandNames().getInfo()) # ['R', 'G', 'B', 'NDWI', 'NDVI']

In [12]:
# Sentinel 2 
s2 = s2Coll.filterDate('2022-06-01','2023-06-01').filterBounds(nrbcBBox).median().select(['B4','B3','B2','B8'],['R','G','B','NIR']).clip(nrbcBBox)

## View Layers in map

In [ ]:
# Create a map
Map = geemap.Map()

# Add the Dynamic World image to the map
Map.addLayer(dw, dwVis, 'Dynamic World')

# Add the Sentinel 2 image to the map
Map.addLayer(s2, visParamsS2, 'Sentinel 2')

# Add the first NICFI image to the map
Map.addLayer(nd_mosaic_2022.first(), visParamsW, 'NICFI Image')

# Center the map at the displayed layers
Map.centerObject(nrbcBBox, 12)

# Display the map
Map

In [25]:
scale = nd_mosaic_2022.first().toFloat().projection().nominalScale()
projection = nd_mosaic_2022.first().toFloat().projection()

In [53]:
# Load a basemap (e.g., a Landsat image)
basemap = s2.select(['R', 'G', 'B']).visualize(bands=['R','G','B'],min=1000,max=4000) # Adjust visualization parameters as needed

# Assuming 'imageCollection' is your collection of monthly images
labeledImages = nd_mosaic_2022.map(lambda image: blendImages(image, basemap))

In [44]:
# Set the export parameters
export_params = {
    'region': geometry,
    'scale': scale,
    'crs': projection,
    'folder': 'EEImageExports',
    'framesPerSecond': 1,
}

# Export the labeledImages as a video
ee.batch.Export.video.toDrive(labeledImages, **export_params).start()

In [54]:
import datetime

# Get the list of dates
dates = labeledImages.aggregate_array('system:time_start').getInfo()

# Convert the dates to the desired format
formatted_dates = [datetime.datetime.fromtimestamp(date/1000).strftime('%B-%Y') for date in dates]

# Print the formatted dates
for date in formatted_dates:
    print(date)

June-2022
July-2022
August-2022
September-2022
October-2022
November-2022
December-2022
January-2022
February-2022
March-2022
April-2022
May-2022


## Add labels to video

In [55]:
import cv2
from moviepy.editor import ImageSequenceClip
import os

In [56]:
# Function to add month and year to the frames
def label_frames(video_path, output_path, start_month, start_year):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file")

    frames = []
    months = ["January", "February", "March", "April", "May",
              "June","July", "August", "September", "October", "November", "December"]

    for frame_num in range(12):  # Assuming 12 frames, one for each month
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate month and year
        month_index = (start_month - 1 + frame_num) % 12
        year = start_year + (start_month - 1 + frame_num) // 12
        label = f"{months[month_index]} {year}"

        # Add label to the frame
        cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        frames.append(frame)

    cap.release()

    # Compile frames into a video
    clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=1)
    clip.write_videofile(output_path, codec='libx264')

In [57]:
# Usage
video_path = "C:\\Users\\ATREE\\Desktop\\myExportVideoTask.mp4"
output_path = "C:\\Users\\ATREE\\Desktop\\myExportVideoTaskLabeled.mp4"
label_frames(video_path, output_path, start_month=6, start_year=2022)

Moviepy - Building video C:\Users\ATREE\Desktop\myExportVideoTaskLabeled.mp4.
Moviepy - Writing video C:\Users\ATREE\Desktop\myExportVideoTaskLabeled.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\ATREE\Desktop\myExportVideoTaskLabeled.mp4
